In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile, execute
from qiskit.quantum_info.operators import Operator, Pauli
import qiskit
from qiskit.extensions import HamiltonianGate, Initialize
from qiskit.visualization import plot_state_city, plot_histogram
from qiskit import quantum_info
from qiskit.quantum_info import Statevector, random_unitary, random_hermitian
from qiskit.circuit.library import PauliEvolutionGate
from qiskit import Aer
from tqdm import tqdm 
from sklearn.linear_model import LogisticRegression
backend = Aer.get_backend('statevector_simulator')

In [ ]:
def input_state(i):
    if i%4==0:
        return np.array([1,0], dtype=complex)
    elif i%4==1:
        return np.array([1,0], dtype=complex)
    elif i%4==2:
        return np.array([1,0], dtype=complex)
    elif i%4==3:
        return np.array([0,1], dtype=complex)
    else:
        print('FAIL')

In [ ]:
n_qubits = 4
n_steps = 400
shots = 10000
op = random_unitary(2**n_qubits, )



In [ ]:
target = np.argmax(np.array([input_state(i) for i in range(n_steps)]), axis=1)

circ = QuantumCircuit(n_qubits, n_steps)
for i in range(n_qubits):
    circ.h(i)
for i in range(n_steps):
    circ.append(op, [i for i in range(n_qubits)])
    circ.measure(0,i)
    circ.initialize(input_state(i), 0)
# circ.draw('mpl')

In [ ]:
simulator = Aer.get_backend('aer_simulator_statevector')
circ = transpile(circ, simulator)
result = simulator.run(circ, shots=shots, memory=True).result()

In [ ]:
counts = result.get_memory()

In [ ]:
res = np.zeros((shots, n_steps))
for i, c in enumerate(counts):
    for j in range(n_steps):
        res[i,j] = float(c[j])
mean = np.mean(res, axis=0)

In [ ]:
mean

In [ ]:
x = 5*(mean-np.mean(mean)) + np.mean(mean)
# x = 2 * mean

In [ ]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity

plt.figure()
for t in np.unique(target):
    mask = t==target
    plt.plot(np.arange(n_steps)[mask], x[mask], marker='o', lw=0)
plt.ylim(-0.1, 1.1)
plt.show()

x_plot = np.linspace(-0.5, 1.5, 10000, endpoint=True)

plt.figure()
for i, t in enumerate(np.unique(target)):
    color = 'C'+str(i)
    mask = t==target
    plt.hist(x[mask], density=True, bins=np.linspace(-0.1, 1.1, 100, endpoint=True),
             histtype='step', color=color)
    kde = KernelDensity(bandwidth=1.5e-2, kernel='gaussian')
    kde.fit(x[mask].reshape(-1, 1))
    logprob = kde.score_samples(x_plot.reshape(-1, 1))
    plt.fill_between(x_plot, np.exp(logprob), alpha=0.5, color=color)
    
plt.xlim(-0.1, 1.1)
plt.show()

In [ ]:
X = x.reshape(-1, 1)
y = target

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline


clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X, y)

clf.score(X_test, y_test)

In [ ]:
target.shape

In [ ]:
target = np.argmax(np.array([input_state(i) for i in range(n_steps)]), axis=1)

circ = QuantumCircuit(n_qubits, n_steps)
for i in range(n_qubits):
    circ.h(i)
for i in range(5):
    circ.append(op, [i for i in range(n_qubits)])
    circ.measure(0,i)
    circ.initialize(input_state(i), 0)
circ.draw('mpl')